In [1]:
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")
using PyCall
using Distributions
unshift!(PyVector(pyimport("sys")["path"]), "")
unshift!(PyVector(pyimport("sys")["path"]), "..")
unshift!(PyVector(pyimport("sys")["path"]), "../Ad05RTheta1RSteer05Abs0FR200")
unshift!(PyVector(pyimport("sys")["path"]), "../CommonFiles")
@pyimport python2juliaReuse as p2j

In [2]:
# policies = ["Baseline/Rewards/b_15e3_3","Retrain/Rewards/rarl","Retrain/Rewards/rarlP10",
#             "FSP/Rewards/nBnoTR","FSP/Rewards/nBnoTRP10"]

In [3]:
pc = p2j.PolicyContainer()
path = "Data/Sep1/Ad05RTheta1RSteer05Abs0FR200/"
# policies = readdir(string(path,"Policy"))
# pnames = []
# for i = 1:length(policies)
#     if policies[i] != ".DS_Store"
#         push!(pnames,policies[i][1:length(policies[i])-4])
#     end
# end
# println(pnames)
pnames = ["b_10e3","rarl_1e3","fsp_1e3"]
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")


2018-09-05 13:46:30.145521: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-05 13:46:30.145535: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-09-05 13:46:30.145539: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-05 13:46:30.145542: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
05-Sep 13:46:30:WARNING:root:replacing docs for 'close :: Tuple{ZMQTransport}' in module 'Main'.
05-Sep 13:46:30

run_env_server_rarl (generic function with 1 method)

In [4]:
mutable struct Counter
    i::Int
    I::Int
    num_collision::Int
    rate_collision::Float64
    total_reward::Float64
    single_reward::Float64
    avg_reward::Float64
    max_reward::Float64
    min_reward::Float64
    rewards::Array{Float64}
    done::Bool
    direction::Int
    total_speed::Float64
    single_speed::Float64
    avg_speed::Float64
    speeds::Array{Float64}
    single_time::Float64
    total_time::Float64
    time_between_collisions::Float64
    total_reward_d::Float64
    single_reward_d::Float64
    avg_reward_d::Float64
    rewards_d::Array{Float64}
end
function Counter(N::Int)
    return Counter(0,N,0,0.0,0.0,0.0,0.0,-Inf,Inf,zeros(N),false,-1,
        0.0,0.0,0.0,zeros(N),0.0,0.0,0.0,0.0,0.0,0.0,zeros(N))
end

Counter

In [7]:
function handle_action_max!(pc,scene,models,roadway,counter,rarl,reverse)
    state = get_observation(scene,models,roadway)
    if rarl
#         action1_arr = pc[:getAction_just](state)
        dist = pc[:getActionDistribution_just](state)
        action1_arr = dist["mean"]
    else
        action1_arr = pc[:getAction_just_baseline](state)
    end
    action1_arr = tanh.(action1_arr)

    dacc = 1.0
    dsteer = 1.0
    if reverse
        dacc = -dacc
        dsteer = -dsteer
    end
    action = Egoaction(action1_arr[1],action1_arr[2],dacc,dsteer)
    
    done, bump = simulate_action!(action,scene,models,roadway)
    counter.single_time += TIMESTEP
    counter.total_time += TIMESTEP
    reward = reward_fn(action,done,scene,models,roadway)
    if bump
        counter.num_collision += 1
    else
        counter.total_reward_d += reward
        counter.single_reward_d += reward
    end
    
    counter.total_reward += reward
    counter.single_reward += reward
    
    speed = scene[1].state.v
    counter.single_speed += speed
    counter.total_speed += speed
    
    if done
        pc[:resetPolicy](1)
        rand_ego!(scene,models,roadway)
        
        counter.i += 1
        if counter.single_reward > counter.max_reward
            counter.max_reward = counter.single_reward
        end
        if counter.single_reward < counter.min_reward
            counter.min_reward = counter.single_reward
        end
        if counter.i > counter.I
            counter.done = true
            counter.avg_reward = counter.total_reward/counter.I
            counter.rate_collision = counter.num_collision/counter.I
            counter.avg_reward_d = counter.total_reward_d/counter.I
            counter.avg_speed = counter.total_speed/(counter.total_time/TIMESTEP)
            counter.time_between_collisions = counter.total_time/counter.num_collision
        else
            counter.rewards[counter.i] = counter.single_reward
            counter.rewards_d[counter.i] = counter.single_reward_d
            counter.speeds[counter.i] = counter.single_speed/(counter.single_time/TIMESTEP)
        end
        counter.single_reward = 0.0
        counter.single_reward_d = 0.0
        counter.single_speed = 0.0
        counter.single_time = 0.0
    end
    
    
end

handle_action_max! (generic function with 1 method)

In [8]:
log_path = string(path,"Max/")
scene, models, roadway = initialize_env()

for pname in pnames
    println(pname)
    policy_path = string(path,"Policy/",pname,".pkl")
    pc[:reset_policy](policy_path=policy_path)
    N = 5

    counter = Counter(N)
    rand_ego!(scene,models,roadway)
    pc[:resetPolicy](1)
    while !counter.done
        handle_action_max!(pc,scene,models,roadway,counter,true,false)
    end
    println(counter.avg_reward)
    println(counter.rate_collision)
    
    writedlm(string(log_path,pname,"_rewards",".txt"),counter.rewards)
    writedlm(string(log_path,pname,"_rewards_d",".txt"),counter.rewards_d)
    writedlm(string(log_path,pname,"_speeds",".txt"),counter.speeds)
    writedlm(string(log_path,pname,"_collision_rate",".txt"),counter.rate_collision)

    counter2 = Counter(N)
    rand_ego!(scene,models,roadway)
    pc[:resetPolicy](1)
    while !counter2.done
        handle_action_max!(pc,scene,models,roadway,counter2,true,true)
    end
    println(counter2.avg_reward)
    println(counter2.rate_collision)

    writedlm(string(log_path,pname,"_rewards2",".txt"),counter2.rewards)
    writedlm(string(log_path,pname,"_rewards_d2",".txt"),counter2.rewards_d)
    writedlm(string(log_path,pname,"_speeds2",".txt"),counter2.speeds)
    writedlm(string(log_path,pname,"_collision_rate2",".txt"),counter2.rate_collision)
end

b_10e3
563.7718662638043
0.0
439.02277326128404
0.0
rarl_1e3
575.7598226107929
0.0
434.0355221834726
0.0
fsp_1e3
566.9017735092768
0.0
6.260580256461009
0.0
